In [6]:
import numpy as numpy
import pandas as pandas

## Выборка

In [7]:
data = {
    'Песок': numpy.random.normal(69, 7, 12),
    'Цемент': numpy.random.normal(123, 17, 12),
    'Щебенка': numpy.random.normal(77, 7, 12),
    'Земля': numpy.random.normal(50, 5, 12),
    'Вода': numpy.random.normal(100, 15, 12),
    'Гравий': numpy.random.normal(95, 9, 12),
    'Кирпичи': numpy.random.normal(33, 4, 12),
    'Газоблоки': numpy.random.normal(111, 21, 12),
    'Доски': numpy.random.normal(112, 22, 12),
    'Сваи': numpy.random.normal(70, 15, 12),
}

In [8]:
table = pandas.DataFrame(data)
table

,Песок,Цемент,Щебенка,Земля,Вода,Гравий,Кирпичи,Газоблоки,Доски,Сваи
0,66.936179,138.396505,84.951897,48.866108,76.553348,89.111231,35.618514,140.666553,109.810635,65.258720
1,64.104674,97.950796,76.461902,48.877165,105.183874,97.269228,35.689294,86.298616,131.687999,86.096961
2,78.106334,105.822337,81.009649,43.984949,98.804944,102.572908,30.321690,84.634785,93.931983,66.418915
3,65.413184,96.422443,74.728767,47.566265,115.296575,91.170127,32.550738,130.879373,131.799536,75.233501
4,66.520862,114.590718,64.334237,50.126771,115.649375,88.942360,35.410876,100.136669,132.852552,60.526254
5,71.025691,115.870548,76.255566,46.850732,108.428978,86.094479,27.383789,116.498653,72.366492,71.025585
6,72.126488,116.575834,81.872946,52.967350,82.002087,113.260167,31.870214,146.555585,101.949591,53.264113
7,67.824158,112.770473,69.423235,46.899027,105.934706,91.943520,28.491083,126.563059,117.973326,61.417385
8,71.137417,131.173936,69.992439,48.166947,99.912686,105.695557,32.467500,70.692090,118.905548,76.485438
9,71.386560,107.299339,76.251013,50.372364,100.919644,91.089781,34.622841,95.817793,79.949969,81.498311


## Получение p0

In [9]:
p0 = table.sum() / table.shape[0]
p0

Песок         69.464207
Цемент       117.338473
Щебенка       75.796389
Земля         48.139080
Вода         100.484333
Гравий        98.118221
Кирпичи       32.746996
Газоблоки    108.526260
Доски        106.719839
Сваи          72.487965
dtype: float64

## Получение среднеквадратичного отклонения

In [10]:
sr = ((table - p0) ** 2).sum() / (table.shape[0] - 1)
sr = sr ** (1 / 2)

## Значение планируемого показателя на 13-ый месяц

In [11]:
table2 = p0 + numpy.random.normal(0, sr, len(p0))
columns = table2.index
pandas.concat([table, pandas.DataFrame([table2], columns=columns)]).reset_index(drop=True)
table2

Песок         65.753872
Цемент       121.578611
Щебенка       74.231374
Земля         42.359353
Вода          93.491863
Гравий        91.255555
Кирпичи       30.882569
Газоблоки    120.937035
Доски         89.848868
Сваи          73.359246
dtype: float64

## Оценки

In [12]:
reliability = sr / p0
reliability

Песок        0.053797
Цемент       0.131384
Щебенка      0.075674
Земля        0.049780
Вода         0.116477
Гравий       0.107105
Кирпичи      0.090368
Газоблоки    0.230648
Доски        0.225880
Сваи         0.157859
dtype: float64

### Уловие 1

In [13]:
cond1 = ((table - p0) < 2 * sr).all()
cond1

Песок        False
Цемент        True
Щебенка       True
Земля        False
Вода          True
Гравий       False
Кирпичи       True
Газоблоки     True
Доски         True
Сваи          True
dtype: bool

### Условие 2

In [14]:
cond2 = p0 > 2 * sr
cond2

Песок        True
Цемент       True
Щебенка      True
Земля        True
Вода         True
Гравий       True
Кирпичи      True
Газоблоки    True
Доски        True
Сваи         True
dtype: bool

### Условие 3

In [15]:
cond3 = (table > 0).all()
cond3

Песок        True
Цемент       True
Щебенка      True
Земля        True
Вода         True
Гравий       True
Кирпичи      True
Газоблоки    True
Доски        True
Сваи         True
dtype: bool

### Цветовая классификация

In [16]:
colClas = pandas.DataFrame({'keyword':table.columns, 
                         'color': 'green'})
i = 0
table.columns
for name in table.columns:
    if (not cond3[name]):
        colClas.color[i] = 'red'
    elif (not cond1[name] and not cond2[name]):
        colClas.color[i] = 'orange'
    elif (not cond1[name] or not cond2[name]):
        colClas.color[i] = 'yellow'
    i += 1
    
new_dict = pandas.Series(colClas.color.values,index=colClas.keyword).to_dict()
colClas.style.applymap(lambda v: f"background-color: {new_dict.get(v, 'None')}")


,keyword,color
0,Песок,yellow
1,Цемент,green
2,Щебенка,green
3,Земля,yellow
4,Вода,green
5,Гравий,yellow
6,Кирпичи,green
7,Газоблоки,green
8,Доски,green
9,Сваи,green
